In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.linear_model import LinearRegression
import scipy.stats as stats

import sys
sys.path.insert(0, "../src/")
import util as util

%autosave 5

In [ ]:
df = pd.read_csv('../Data/EvolyticsDataScienceExercise.csv')
revenueCols = [col for col in df.columns if 'purchase_revenue' in str(col)]
revenuedf = df[revenueCols]
df[orderRevenue] = revenuedf.sum(axis=1)

# Univariate Analysis

In [ ]:
datetimelikecolumns = ['date','min_timestamp_date','min_timestamp_time','max_timestamp','date.1','upsell_timestamp_date','upsell_timestamp_time']
catagoricalCols = []

n = df.shape[0]
dateColumns = []
for col in df.columns:
    col = str(col)
    print(col)
    parsedValues = [util.tryParse(x)[1] for x in df[col] if util.tryParse(x)[0]]
    
    if col in datetimelikecolumns:
        sample = df[df[col] != "(null)"].dropna()[col]
        sample =  [x for x in pd.to_datetime(sample)]
        n_nans = n - len(sample)
        print(f'{n_nans} nans found. {n_nans*100.0/n}% of data is nan')
        fig = plt.figure()
        plt.hist(sample)
        plt.xticks(rotation=90)
        plt.show()
    elif (len(parsedValues) > 0 ) and (len(list(set(df[col]))) > 15):
        sample = np.array(parsedValues)
        n_nans = n - len(sample)
        print(f'{n_nans} nans found. {n_nans*100.0/n}% of data is nan')
        print(f'{np.mean(sample)} +/- {np.std(sample)}')
        fig = plt.figure()
        plt.hist(sample)
        plt.show()
    else:
        categoricalCols.append(col)
        print(df[col].value_counts())
    

# Correlation Analysis

In [ ]:
corrDictionary = {
    "name":[],
    "iCol":[],
    "jCol":[],
    "r":[],
    "absr":[],
    "p":[],
    "m":[],
    "b":[]
}
dfCopy = df.copy()
for dtcol in datetimelikecolumns:
    dfCopy[dtcol] = pd.to_datetime()

for catcol in catagoricalCols:
    dummyDF = pd.get_dummies(dfCopy[catcol])
    for dummyCol in dummyDF.columns:
        dfCopy[dummyCol] = dummyDF[dummyCol]
    dfCopy.drop(catcol,axis=1,inplace=True)
    
for i,iCol in enumerate(dfCopy.columns):
    for j,jCol in enumerate(dfCopy.columns):
        if i < j:
            corrDF = dfCopy[[iCol,jCol]].dropna()
            iList = list(np.array(corrDF[iCol]).reshape(-1))
            jList = list(np.array(corrDF[jCol]).reshape(-1))
            r,p = stats.spearmanr(iList,jList)
            if p < 0.05:
                corrDictionary["name"] += f"{jCol} vs {iCol}"
                corrDictionary["iCol"] += iCol 
                corrDictionary["jCol"] += jCol 
                corrDictionary["r"] += r
                corrDictionary["absr"] += abs(r)
                corrDictionary["p"] += p
                lm = LinearRegression()
                lm.fit(corrDF[iCol],corrDF[jCol])
                corrDictionary["m"] += lm.coef_[0]
                corrDictionary["b"] += lm.intercept_
                

In [ ]:
df[df["date"] != df["date.1"]]